# Exploratory Data Analysis

## Buildboard Charts
link: https://www.kaggle.com/datasets/dhruvildave/billboard-the-hot-100-songs

In [1]:
import pandas as pd
import numpy as np
import os
import kaggle

### Get data from Kaggle
To download csv file using Python, you will need a [Kaggle API Key](https://www.kaggle.com/docs/api). 

Make sure to save the API key in 'C:\Users\USER_NAME.kaggle' directory.

In [2]:
kaggle.api.authenticate()

In [3]:
kaggle.api.dataset_download_files('dhruvildave/billboard-the-hot-100-songs', 'billboard_data', unzip=True)

### Set up directory

In [4]:
ROOT = os.getcwd()
print(f"Root Directory: {ROOT}")

BILLBOARD_DIR = os.path.join(ROOT, 'billboard_data')
print(f"Bilboard Directory: {BILLBOARD_DIR}")

Root Directory: C:\Users\Nicholas\Desktop\Masters - Classes\MSDS455\MSDS455-The_Show
Bilboard Directory: C:\Users\Nicholas\Desktop\Masters - Classes\MSDS455\MSDS455-The_Show\billboard_data


### Read into pandas data frame

In [5]:
billboard_df = pd.read_csv(os.path.join(BILLBOARD_DIR, 'charts.csv'))
print(len(billboard_df))
billboard_df.head()

330087


,date,rank,song,artist,last-week,peak-rank,weeks-on-board
0,2021-11-06,1,Easy On Me,Adele,1.0,1,3
1,2021-11-06,2,Stay,The Kid LAROI & Justin Bieber,2.0,1,16
2,2021-11-06,3,Industry Baby,Lil Nas X & Jack Harlow,3.0,1,14
3,2021-11-06,4,Fancy Like,Walker Hayes,4.0,3,19
4,2021-11-06,5,Bad Habits,Ed Sheeran,5.0,2,18


In [6]:
billboard_df.describe()

,rank,last-week,peak-rank,weeks-on-board
count,330087.000000,297775.000000,330087.000000,330087.000000
mean,50.500929,47.591631,40.970629,9.161785
std,28.866094,28.054360,29.347481,7.618264
min,1.000000,1.000000,1.000000,1.000000
25%,26.000000,23.000000,13.000000,4.000000
50%,51.000000,47.000000,38.000000,7.000000
75%,76.000000,72.000000,65.000000,13.000000
max,100.000000,100.000000,100.000000,90.000000


In [7]:
billboard_df.dtypes

date               object
rank                int64
song               object
artist             object
last-week         float64
peak-rank           int64
weeks-on-board      int64
dtype: object

In [8]:
# Check for Nulls
print(billboard_df.isnull().sum())

date                  0
rank                  0
song                  0
artist                0
last-week         32312
peak-rank             0
weeks-on-board        0
dtype: int64


In [9]:
# Unique tracks
print(f"Number of unique tracks: {len(billboard_df['song'].unique())}")

Number of unique tracks: 24620


#### Clean up date column from dtype Object to Datetime and create new column for "Year" and "Month"

In [10]:
billboard_df['date'] = pd.to_datetime(billboard_df['date'])
billboard_df['year'] = billboard_df['date'].dt.year
billboard_df['month'] = billboard_df['date'].dt.month

billboard_df.head()

,date,rank,song,artist,last-week,peak-rank,weeks-on-board,year,month
0,2021-11-06,1,Easy On Me,Adele,1.0,1,3,2021,11
1,2021-11-06,2,Stay,The Kid LAROI & Justin Bieber,2.0,1,16,2021,11
2,2021-11-06,3,Industry Baby,Lil Nas X & Jack Harlow,3.0,1,14,2021,11
3,2021-11-06,4,Fancy Like,Walker Hayes,4.0,3,19,2021,11
4,2021-11-06,5,Bad Habits,Ed Sheeran,5.0,2,18,2021,11


In [11]:
# Look at unique years
sorted(billboard_df['year'].unique())

[1958,
 1959,
 1960,
 1961,
 1962,
 1963,
 1964,
 1965,
 1966,
 1967,
 1968,
 1969,
 1970,
 1971,
 1972,
 1973,
 1974,
 1975,
 1976,
 1977,
 1978,
 1979,
 1980,
 1981,
 1982,
 1983,
 1984,
 1985,
 1986,
 1987,
 1988,
 1989,
 1990,
 1991,
 1992,
 1993,
 1994,
 1995,
 1996,
 1997,
 1998,
 1999,
 2000,
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
 2020,
 2021]

In [20]:
# track name list
track_ls = list(billboard_df['song'].unique())
print(track_ls[:5])

# artist name list
artist_ls = list(billboard_df['artist'].unique())
print(artist_ls[:5])

['Easy On Me', 'Stay', 'Industry Baby', 'Fancy Like', 'Bad Habits']
['Adele', 'The Kid LAROI & Justin Bieber', 'Lil Nas X & Jack Harlow', 'Walker Hayes', 'Ed Sheeran']


## Spotify API
- [Spotipy Documentation](https://spotipy.readthedocs.io/en/2.22.1/)

In [13]:
import json
import requests
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import config

In [14]:
client_credentials_manager = SpotifyClientCredentials(client_id=config.client_id, client_secret=config.client_secret)

In [17]:
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [80]:
artist= artist_ls[0]
track= track_ls[0]

request = sp.search(q='artist:' + artist + ' track:' + track, type='track')

In [86]:
track_id = request['tracks']['items'][0]['id']
print(f"Track ID: {track_id}")

Track ID: 0gplL1WMoJ6iYaPgMCL0gX


In [94]:
# Album name
album_name = request['tracks']['items'][1]['album']['name']
print(f"Album name: {album_name}")

Album name: 30


In [85]:
# Get audio features
sp.audio_features(track_id)

[{'danceability': 0.604,
  'energy': 0.366,
  'key': 5,
  'loudness': -7.519,
  'mode': 1,
  'speechiness': 0.0282,
  'acousticness': 0.578,
  'instrumentalness': 0,
  'liveness': 0.133,
  'valence': 0.13,
  'tempo': 141.981,
  'type': 'audio_features',
  'id': '0gplL1WMoJ6iYaPgMCL0gX',
  'uri': 'spotify:track:0gplL1WMoJ6iYaPgMCL0gX',
  'track_href': 'https://api.spotify.com/v1/tracks/0gplL1WMoJ6iYaPgMCL0gX',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0gplL1WMoJ6iYaPgMCL0gX',
  'duration_ms': 224695,
  'time_signature': 4}]

## Metacritic Webscrape
URL: https://www.metacritic.com/

**EXAMPLE**

![example1](images/metacritic1.png)

In [107]:
from bs4 import BeautifulSoup

user_agent = {'User-agent': 'Mozilla/5.0'}

url = f"https://www.metacritic.com/music/{album_name}/{artist}"

r = requests.get(search_url, headers = user_agent)
soup = BeautifulSoup(r.text,"html.parser") 

In [111]:
review_data = soup.find_all('div', class_='metascore_w xlarge album positive')
review_data

[<div class="metascore_w xlarge album positive"><meta content="0" itemprop="worstRating"/><meta content="100" itemprop="bestRating"/><span itemprop="ratingValue">88</span></div>]

In [118]:
rating = review_data[0].find_all('span')[0].text

print(f"Album rating from metacritic: {rating}")

Album rating from metacritic: 88
